# El Método de Benjamini-Hochberg (BH): Controlando el FDR ✨

---

## **# 1. El Dilema de Bonferroni y la Tasa de Descubrimiento Falso (FDR)**

En el *notebook* anterior, aprendimos que la **Corrección de Bonferroni** controla la **Tasa de Error Familiar (FWER)**, asegurando que la probabilidad de cometer *al menos un* Falso Positivo ($\text{Error Tipo I}$) en todo el conjunto de pruebas sea $\le \alpha$. El costo de esto es que se vuelve **demasiado conservadora**, lo que aumenta los Falsos Negativos ($\text{Error Tipo II}$) y hace que el modelo pierda potencia estadística.

El método de **Benjamini-Hochberg (BH)** resuelve esto introduciendo un concepto más flexible: la **Tasa de Descubrimiento Falso (FDR)**.

### **FWER vs. FDR**

| Tasa de Error | Definición | Nivel de Control |
| :--- | :--- | :--- |
| **FWER** (Bonferroni) | Probabilidad de cometer **al menos un** falso positivo. | **Muy estricto.** Ideal para pruebas críticas (ej. medicina). |
| **FDR** (Benjamini-Hochberg) | **Proporción esperada** de Falsos Positivos entre todos los resultados que declaramos como significativos. | **Más flexible.** Ideal para *data mining* y exploración. |

**Objetivo de BH:** Si fijamos el $\text{FDR}$ en 0.05, solo esperamos que, en promedio, el 5% de los resultados que etiquetamos como "significativos" sean en realidad Falsos Positivos.

---

## **# 2. El Algoritmo de Benjamini-Hochberg (Paso a Paso)**

El método BH no ajusta directamente el nivel $\alpha$, sino que compara cada p-valor individual con un **umbral variable** que depende del *ranking* de dicho p-valor.

Supongamos que realizamos $m$ pruebas (ej. $m=20$ *lags* en Ljung-Box) y fijamos el nivel de control $\text{FDR}$ deseado en $Q$ (ej. $Q=0.05$).

### **Pasos del Algoritmo**

1.  **Obtener p-valores:** Calcula el p-valor ($p_i$) para cada una de las $m$ pruebas.
2.  **Ordenar ($p$-ranking):** Ordena los $m$ p-valores de menor a mayor.
    * $p_{(1)} \le p_{(2)} \le \dots \le p_{(m)}$
3.  **Calcular el Umbral Crítico:** Para cada p-valor ordenado $p_{(i)}$, calcula su **valor crítico** (umbral de significancia) usando la siguiente fórmula:

$$
\text{Umbral Crítico } = \frac{i}{m} \times Q
$$

Donde:
* $i$: El *rank* (posición) del p-valor ordenado (de 1 a $m$).
* $m$: El número total de pruebas.
* $Q$: El nivel $\text{FDR}$ deseado (ej. 0.05).

4.  **Encontrar el Límite ($k$):** Recorre la lista de p-valores ordenados, comenzando por el más grande ($i=m$) hacia el más pequeño ($i=1$). El límite $k$ es el **p-valor de mayor *rank* ($i$)** que cumple la siguiente condición:

$$
p_{(i)} \le \frac{i}{m} \times Q
$$

5.  **Decisión Final:** Declara como significativos (Rechaza $H_0$) a **todos los p-valores desde $p_{(1)}$ hasta $p_{(k)}$**.

---

## **# 3. Implementación Conceptual en Python (Simulación)**

Este ejemplo simula los p-valores de 10 pruebas y aplica tanto Bonferroni como Benjamini-Hochberg para demostrar la diferencia en potencia.

In [1]:
import pandas as pd
import numpy as np

# 1. Parámetros
m_pruebas = 10       # Total de lags (m)
FDR_deseado = 0.05   # Nivel Q

# 2. P-valores simulados (Ejemplo de 10 lags, desordenados)
p_valores_brutos = np.array([0.045, 0.003, 0.150, 0.001, 0.002, 0.060, 0.048, 0.200, 0.0005, 0.300])

# 3. Organizar los datos en un DataFrame para el BH
df_resultados = pd.DataFrame({
    'p_valor': p_valores_brutos
})

# 4. Ordenar p-valores (Paso 2)
df_resultados = df_resultados.sort_values(by='p_valor').reset_index(drop=True)
df_resultados['i_rank'] = df_resultados.index + 1  # Rank (i) de 1 a m

# 5. Calcular los Umbrales (Paso 3)
df_resultados['Umbral_Critico_BH'] = (df_resultados['i_rank'] / m_pruebas) * FDR_deseado
df_resultados['Umbral_Bonferroni'] = FDR_deseado / m_pruebas # Bonferroni es estático

# 6. Encontrar el Límite k (Paso 4)
# Crear una columna booleana para la condición p(i) <= (i/m)*Q
df_resultados['Es_Significativo_BH'] = df_resultados['p_valor'] <= df_resultados['Umbral_Critico_BH']

# El índice k es el MÁXIMO rank que cumple la condición.
k_limite = df_resultados[df_resultados['Es_Significativo_BH']]['i_rank'].max()

if np.isnan(k_limite):
    k_limite = 0

print(f"Nivel de control FDR (Q): {FDR_deseado}")
print(f"Umbral de Bonferroni (0.05/10): {df_resultados['Umbral_Bonferroni'].iloc[0]:.4f}")
print(f"Límite K encontrado por BH: {int(k_limite)}")

print("\nResultados de Comparación (BH vs. Bonferroni):")
print("-" * 60)
print(df_resultados[['i_rank', 'p_valor', 'Umbral_Critico_BH', 'Umbral_Bonferroni']])
print("-" * 60)

# 7. Decisión Final
p_valores_bh_significativos = df_resultados[df_resultados['i_rank'] <= k_limite]
p_valores_bonf_significativos = df_resultados[df_resultados['p_valor'] <= df_resultados['Umbral_Bonferroni'].iloc[0]]

print(f"\nTotal de Significativos (Bonferroni): {len(p_valores_bonf_significativos)}")
print(f"Total de Significativos (Benjamini-Hochberg): {len(p_valores_bh_significativos)}")

Nivel de control FDR (Q): 0.05
Umbral de Bonferroni (0.05/10): 0.0050
Límite K encontrado por BH: 4

Resultados de Comparación (BH vs. Bonferroni):
------------------------------------------------------------
   i_rank  p_valor  Umbral_Critico_BH  Umbral_Bonferroni
0       1   0.0005              0.005              0.005
1       2   0.0010              0.010              0.005
2       3   0.0020              0.015              0.005
3       4   0.0030              0.020              0.005
4       5   0.0450              0.025              0.005
5       6   0.0480              0.030              0.005
6       7   0.0600              0.035              0.005
7       8   0.1500              0.040              0.005
8       9   0.2000              0.045              0.005
9      10   0.3000              0.050              0.005
------------------------------------------------------------

Total de Significativos (Bonferroni): 4
Total de Significativos (Benjamini-Hochberg): 4
